#    Compilers and Languages
## Programming assignment : Part 1 : input regex -> NFA

Table of Contents:
1. [The Shunting Yard Algorithm](#1.-Introduction---Shunting-Yard-Algorithm)


# The shunting yard algorithm (infix to postfix)

### Introduction
The shunting yard algorithm is a method for parsing mathematical expressions specified in infix notation. It can be used to produce output in Reverse Polish Notation (RPN) or as an abstract syntax tree (AST). The algorithm was invented by Edsger Dijkstra and named the "shunting yard" algorithm because its operation resembles that of a railroad shunting yard.

We will use the shunting yard algorithm to convert the regular expression to postfix notation, which will be used to create the NFA.

### Goal
converting something like `(A+B) * (C | D)` to `A+B . * CD |.`

### Features
. No recursion => Iterative approach.
. No trees are needed.
. No Ambiguities.
. Handles precedence of operators.

### Data Structures
1. infix (input regex) : The regular expression in infix notation (string)
   . Note : infix means the operator is between the operands. eg: `A+B` (The normal way we write expressions)
2. postfix : The regular expression in postfix notation (string)
    . Note : postfix means the operator is after the operands. eg: `AB+` (The way we write expressions in the shunting yard algorithm)
3. stack : The stack used in the algorithm (list)

### Algorithm
0. preprocess the infix to add a `.` between the operands where needed. (eg: `A(B|C)` => `A.(B|C)`). also substitute ranges like `a-z` with the union of all the characters in the range.
1. Create an empty stack for operators. Create an empty list for the output.
2. For each character in the infix :
    - If char is `(` or `[` => push to stack
    - If char is `)` or `]` => pop from stack until nearest matching `(` or `[` , append to output
    - If char is an operator => pop from stack until the top of the stack has a lower precedence than the current operator, then push the current operator to the stack
    - If char is an operand/normal character => append to output
3. Pop all the remaining operators from the stack and append to the output


In [94]:
# Important declarations
# -----------------------
# Dictionary containing the precedence of the operators
regex_operators_precedence = {
    '*' : 5,
    '+' : 4,
    '?' : 3,
    '.' : 2,
    '|' : 1,
    '(' : 0,
    ')' : 0
}

alphanumeric = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
alphabetic = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'


## Preprocessing ...

In [137]:
# Utility functions
# -----------------
def preprocess_regex(regex):
    """
    Preprocess the regex to do the following:
        1-[ranges] add OR's | between in square brackets characters
        2-[ranges] replace dash - with the corresponding sequence of characters with OR's between them
        3-add the missing '.' operators
    """
    #step 1 : add OR's | between in square brackets characters without -
    square_brackets_open = False
    for i in range(len(regex)):
        if regex[i] == '[':
            square_brackets_open = True
        elif regex[i] == ']':
            square_brackets_open = False
        elif square_brackets_open and regex[i] != '-' and regex[i] in alphanumeric and i+1 < len(regex) and regex[i+1] in alphanumeric:
            regex = regex[:i+1] + '|' + regex[i+1:]
    print(f"After step 1 : {regex}")
    #step 2 : replace dash - with the corresponding sequence of characters with OR's between them
    i = 0
    while i < len(regex):
        c = regex[i]
        print(f"i = {i}, c = {c}")
        if c == '-':
            #pop the last element from the stack
            first = regex[i-1] if regex[i-1] in alphanumeric else None
            if not first:
                raise ValueError('Range error: element after - is not alphanumeric')
            #access Char + 1
            last = regex[i + 1] if i+1 < len(regex) and regex[i+1] in alphanumeric else None
            if not last:
                raise ValueError('Range error: element after - is not alphanumeric')
            #throw the - operator from the regex
            operating_list = []
            for char in alphanumeric:
                if alphanumeric.index(char) > alphanumeric.index(first) and alphanumeric.index(char) < alphanumeric.index(last):
                    #append | between the characters
                    # if char != last:
                    operating_list.append('|')
                    operating_list.append(char)
            operating_list.append('|')
            #replace - in regex with the operating list
            #using i to get the index of the first character in the range
            regex = regex[:i] + ''.join(operating_list) + regex[i+1:]
            i += len(operating_list)
        i += 1

    print(f"After step 2 : {regex}")
    #step 3 : add the missing '.' operators
    check1_list = ['*', '+', '?', ')', ']']
    new_regex = ''
    for i,c in enumerate(regex):
        if i > 0 and c in check1_list and i+1 < len(regex) and regex[i+1] not in check1_list:
            new_regex += c + '.'
        elif c in alphanumeric and i+1 < len(regex) and ((regex[i+1] in alphanumeric) or regex[i+1] in ['(', '[']):
            new_regex += c + '.'
        else:
            new_regex += c
    print(f"After step 3 : {new_regex} Preprocessed!")
    return new_regex

In [138]:
def infix_to_postfix(regex):
    """
    Convert an infix regular expression to a postfix regular expression.

    Parameters:
    regex (str): The infix regular expression to convert to postfix.

    Returns:
    str: The postfix regular expression.
    """
    regex = preprocess_regex(regex)
    # Create a stack to hold the operators
    stack = []
    # Create a list to hold the postfix regular expression
    postfix = []
    # Iterate over the characters in the infix regular expression
    for index,char in enumerate(regex):
        #1. '(' or '[': Push it onto the stack
        if char == '(' or char == '[':
            stack.append(char)
        #2. ')' or ']': Pop operators from the stack and append them to the postfix list until '(' or '[' is found
        elif char == ')' :
            while stack[-1] != '(':
                postfix.append(stack.pop())
            #error handling : check empty stack
            if not stack:
                raise ValueError('Parentheses mismatch : found ")" in the stack without "("')
            stack.pop()
        elif char == ']':
            while stack[-1] != '[':
                postfix.append(stack.pop())
            #error handling : check empty stack
            if not stack:
                raise ValueError('Parentheses mismatch : found "]" in the stack without "["')
            stack.pop()
        #3. Operator: Pop operators from the stack and append them to the postfix list until an operator with lower precedence is found
        elif char in regex_operators_precedence:
            #checks:
            # 1. if the stack is not empty
            # 2. if the stack[-1] is an operator => meaning the last element in the stack is an operator
            # 3. if the precedence of the operator in the stack is greater than or equal to the precedence of the current operator
            while stack and stack[-1] in regex_operators_precedence and regex_operators_precedence[stack[-1]] >= regex_operators_precedence[char]:
                postfix.append(stack.pop())
            stack.append(char)
        #else: Append the character to the postfix list
        else:
            # A normal character => append it to the postfix list
            postfix.append(char)
        # empty the stack
    while stack:
        #error handling : check '(' in the stack
        if stack[-1] == '(':
            raise ValueError('Parentheses mismatch : found "(" in the stack without ")"')
        postfix.append(stack.pop())
    return ''.join(postfix)



In [141]:
# test cases
# ----------
def test (id,regex, expected_postfix):
    output =infix_to_postfix(regex)
    print(f"Test case #{id} : {regex} received : {output} expected : {expected_postfix} => Passed")
    assert output == expected_postfix

# test(1,'a+b*c', 'a+b*.c.')
# test(2,'[a-z]+', 'a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z+')
print(infix_to_postfix('[a-zA-Z]+'))
print(infix_to_postfix('[f-p0-9]+(a*bcd?|cde*)'))



print('All test cases passed!')

After step 1 : [a-z|A-Z]+
i = 0, c = [
i = 1, c = a
i = 2, c = -
i = 52, c = |
i = 53, c = A
i = 54, c = -
i = 104, c = ]
i = 105, c = +
After step 2 : [a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z]+
After step 3 : [a|b|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z]+ Preprocessed!
ab|c|d|e|f|g|h|i|j|k|l|m|n|o|p|q|r|s|t|u|v|w|x|y|z|A|B|C|D|E|F|G|H|I|J|K|L|M|N|O|P|Q|R|S|T|U|V|W|X|Y|Z|+
After step 1 : [f-p|0-9]+(a*bcd?|cde*)
i = 0, c = [
i = 1, c = f
i = 2, c = -
i = 22, c = |
i = 23, c = 0
i = 24, c = -
i = 42, c = ]
i = 43, c = +
i = 44, c = (
i = 45, c = a
i = 46, c = *
i = 47, c = b
i = 48, c = c
i = 49, c = d
i = 50, c = ?
i = 51, c = |
i = 52, c = c
i = 53, c = d
i = 54, c = e
i = 55, c = *
i = 56, c = )
After step 2 : [f|g|h|i|j|k|l|m|n|o|p|0|1|2|3|4|5|6|7|8|9]+(a*bcd?|cde*)
After step 3 : [f|g|h|i|j|k|l|m|n|o|p|0|1|2|3|4|5|6|7|8|9]+.(a*.b.c.d?.|c.d.e*) Preprocessed!
fg|

# Postfix to NFA